In [ ]:
from simple_multi_unet_model import * # Uses softmax as last activation
from supporters import *
from keras.utils import normalize
import os
import glob
import cv2
import numpy as np
from matplotlib import pyplot as plt
import SimpleITK as sitk
import imageio
import nibabel as nib
import cv2

### Read train and mask files into numpy array

In [ ]:
def read_volumetric_data():
    train_path = '../data/MM_WHS/train_images'
    mask_path = '../data/MM_WHS/mask_images'
    
    train_files = glob.glob(os.path.join(train_path, '*.nii.gz'))
    mask_files = glob.glob(os.path.join(mask_path, '*.nii.gz'))

    train_images = []
    for index_file in range(3):
        pat = sitk.ReadImage(train_files[index_file], sitk.sitkFloat32)
        pat = sitk.GetArrayFromImage(pat)
        for i in range(pat.shape[0]):
            train_images.append(pat[i])
            
    mask_images = []
    for index_file in range(3):
        pat = sitk.ReadImage(mask_files[index_file], sitk.sitkFloat32)
        pat = sitk.GetArrayFromImage(pat)
        for i in range(pat.shape[0]):
            mask_images.append(pat[i])
        
    train_images = np.array(train_images)
    mask_images = np.array(mask_images)
    return train_images, mask_images


train_images, mask_images = read_volumetric_data()
train_images.shape, mask_images.shape

### Convert Nifti to Folder pngs

In [ ]:
# Load the .nii.gz file
image = sitk.ReadImage("../data/MM_WHS/ct_train/ct_train_1001_image.nii.gz")
pat1001 = sitk.GetArrayFromImage(image)

def convert_nii_to_image():
    (z, x, y) = pat1001.shape 
    
    for i in range(z): 
        img = pat1001[i,:,:]
        img = (img - np.min(img)) / np.ptp(img)  # Normalize the data to 0 - 1
        img = 255 * img  # Now scale by 255
        img = img.astype(np.uint8)
        imageio.imsave(f'../data/MM_WHS/ct_train/ct_train_1001_image/{str(i).zfill(3)}.png', img)
        
convert_nii_to_image()

# Specify the path
path = '../data/MM_WHS/ct_train/ct_train_1001_image'

# Use glob to match the pattern '*.png', and sort the files
files = sorted(glob.glob(os.path.join(path, '*.png')))

train_images = []
for file in files:
    img = cv2.imread(file, 0)   
    train_images.append(img)   
    
train_images = np.array(train_images)

In [ ]:
explore_3D_array_comparison(train_images, pat1001)

### Encoding

In [ ]:
# 8 pixel values -> background + 7 classes
y_res = np.unique(mask_images)
y_res

In [ ]:
# Encode labels... but multi dim array so need to flatten, encode and reshape
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()
n, h, w = mask_images.shape
mask_images_reshaped = mask_images.reshape(-1,1)

mask_images_reshaped_encoded = labelencoder.fit_transform(mask_images_reshaped)
mask_images_encoded_original_shape = mask_images_reshaped_encoded.reshape(n, h, w)
np.unique(mask_images_encoded_original_shape)

In [ ]:
train_images = np.expand_dims(train_images, axis=3)
train_images = normalize(train_images, axis=1)
train_masks_inputs = np.expand_dims(mask_images_encoded_original_shape, axis=3)

train_images.shape, train_masks_inputs.shape

In [ ]:
# after normalization
np.max(train_images), np.min(train_images), np.max(train_masks_inputs), np.min(train_masks_inputs)

In [ ]:
# image after normalization
plt.imshow(train_images[200], cmap="gray")

### Splitting data

In [ ]:
n_classes = 8
'''
Create a subset of data for quick testing
Picking 10% for testing and remaining for training (train set + validation set)
'''

from sklearn.model_selection import train_test_split
X1, X_test, y1, y_test = train_test_split(train_images, train_masks_inputs, test_size = 0.1, random_state = 0)

# Further split training data to a smaller subset for quick testing of models
X_train, X_do_not_use, y_train, y_do_not_use = train_test_split(X1, y1, test_size = 0.2, random_state = 0)

print("Class values in the dataset are ... ", np.unique(y_train))  # 0 is the background/few unlabeled 

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print(np.max(X_train), np.min(X_train), np.max(y_train), np.min(y_train))

In [ ]:
n_classes = 8
from keras.utils import to_categorical
train_masks_cat = to_categorical(y_train, num_classes=n_classes)
y_train_cat = train_masks_cat.reshape((y_train.shape[0], y_train.shape[1], y_train.shape[2], n_classes))

test_masks_cat = to_categorical(y_test, num_classes=n_classes)
y_test_cat = test_masks_cat.reshape((y_test.shape[0], y_test.shape[1], y_test.shape[2], n_classes))

In [ ]:
y_train_cat.shape, y_test_cat.shape

In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(mask_images_reshaped_encoded),
                                                 mask_images_reshaped_encoded)
print("Class weights are...:", class_weights)

### UNet truyền thống bình thường

In [ ]:
IMG_HEIGHT = X_train.shape[1]
IMG_WIDTH  = X_train.shape[2]
IMG_CHANNELS = X_train.shape[3]

def get_model():
    return multi_unet_model(n_classes=n_classes, IMG_HEIGHT=IMG_HEIGHT, IMG_WIDTH=IMG_WIDTH, IMG_CHANNELS=IMG_CHANNELS)

# can replace with focal loss
model = get_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(X_train, y_train_cat, 
                    batch_size = 16, 
                    verbose=1, 
                    epochs=5, 
                    validation_data=(X_test, y_test_cat), 
                    # class_weight=class_weights,   
                    shuffle=False)

In [ ]:
history2 = model.fit(X_train, y_train_cat, batch_size = 16, initial_epoch=5, validation_data=(X_test, y_test_cat), verbose=1, epochs=10, shuffle=False)

In [ ]:
# save the model
model.save('training/models/model_multi_v1.hdf5')

# Evaluate the model
_, acc = model.evaluate(X_test, y_test_cat)
print("Accuracy is = ", (acc * 100.0), "%")

In [ ]:
model.save('training/models/model_multi_v1.h5')

In [ ]:
import tensorflow as tf
loaded_model = tf.keras.models.load_model('training/models/model_multi_v1.h5')

_, acc = loaded_model.evaluate(X_test, y_test_cat)
print("Accuracy is = ", (acc * 100.0), "%")

### Unet kết hợp VGG. Resnet làm BackBones

In [ ]:
import segmentation_models as sm

BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)

# preprocess input
x_train = preprocess_input(X_train)
x_test = preprocess_input(X_test)

# define model
model = sm.Unet(BACKBONE, encoder_weights='imagenet')
model.compile(
    'Adam',
    loss=sm.losses.bce_jaccard_loss,
    metrics=[sm.metrics.iou_score],
)

model.fit(
   x=x_train,
   y=y_train,
   batch_size=16,
   epochs=100,
   validation_data=(x_test, y_test),
)

### Metrics Visualization

In [ ]:
# Visualize metrics
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Mất mát tập huấn luyện')
plt.plot(epochs, val_loss, 'r', label='Mất mát tập kiếm chứng')
plt.title('Mất mát trên tập huấn luyện & kiểm chứng')
plt.xlabel('số lần huấn luyện')
plt.ylabel('Chỉ số mất mát')
plt.legend()
plt.show()

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.plot(epochs, acc, 'y', label='Độ chính xác tập huấn luyện')
plt.plot(epochs, val_acc, 'r', label='Độ chính xác tập kiểm chứng')
plt.title('Độ chính xác trên tập huấn luyện & kiểm chứng')
plt.xlabel('số lần huấn luyện')
plt.ylabel('Chỉ số chính xác')
plt.legend()
plt.show()

### Intersection over Union

In [ ]:
# Load sample and see what output is
sample = sitk.ReadImage('../data/MM_WHS/train_images/ct_train_1006_image.nii.gz', sitk.sitkFloat32)
sample = sitk.GetArrayFromImage(sample)
sample.shape

In [ ]:
sample = np.expand_dims(sample, axis=3)
sample = normalize(sample, axis=1)
sample.shape

In [ ]:
y_pred = loaded_model.predict(sample)
y_pred

In [ ]:
y_pred.shape

In [ ]:
y_pred_argmax = np.argmax(y_pred, axis=3)
y_pred_argmax.shape

In [ ]:
np.max(y_pred_argmax), np.min(y_pred_argmax)

In [ ]:
y_res

In [ ]:
y_res2 = np.array([ 205., 420., 500., 550., 600., 820., 850.])
y_res2

In [ ]:
# use np.take to map the indices to values
y_mapped = np.take(y_res2, y_pred_argmax)

print(y_mapped.shape)
np.max(y_mapped), np.min(y_mapped)

In [ ]:
explore_3D_array(y_mapped)

In [ ]:
plt.imshow(y_mapped[135], cmap='gray')

In [ ]:
model.load_weights('training/models/model_multi_v1.hdf5')  

# IOU (Intersection over UNion)
y_pred = model.predict(X_test)
y_pred_argmax = np.argmax(y_pred, axis=3)

# Using built in keras function
from keras.metrics import MeanIoU
n_classes = 8
IOU_keras = MeanIoU(num_classes=n_classes)  
IOU_keras.update_state(y_test[:,:,:,0], y_pred_argmax)
print("Mean IoU =", IOU_keras.result().numpy())

# To calculate I0U for each class...
values = np.array(IOU_keras.get_weights()).reshape(n_classes, n_classes)
print(values)

class1_IoU = values[0,0] / (values[0,0] + values[0,1] + values[0,2] + values[0,3] + values[1,0]+ values[2,0]+ values[3,0])
class2_IoU = values[1,1] / (values[1,1] + values[1,0] + values[1,2] + values[1,3] + values[0,1]+ values[2,1]+ values[3,1])
class3_IoU = values[2,2] / (values[2,2] + values[2,0] + values[2,1] + values[2,3] + values[0,2]+ values[1,2]+ values[3,2])
class4_IoU = values[3,3] / (values[3,3] + values[3,0] + values[3,1] + values[3,2] + values[0,3]+ values[1,3]+ values[2,3])
class5_IoU = values[3,3] / (values[3,3] + values[3,0] + values[3,1] + values[3,2] + values[0,3]+ values[1,3]+ values[2,3])
class6_IoU = values[3,3] / (values[3,3] + values[3,0] + values[3,1] + values[3,2] + values[0,3]+ values[1,3]+ values[2,3])
class7_IoU = values[3,3] / (values[3,3] + values[3,0] + values[3,1] + values[3,2] + values[0,3]+ values[1,3]+ values[2,3])
class8_IoU = values[3,3] / (values[3,3] + values[3,0] + values[3,1] + values[3,2] + values[0,3]+ values[1,3]+ values[2,3])

print("IoU for class1 is: ", class1_IoU)
print("IoU for class2 is: ", class2_IoU)
print("IoU for class3 is: ", class3_IoU)
print("IoU for class4 is: ", class4_IoU)
print("IoU for class4 is: ", class5_IoU)
print("IoU for class4 is: ", class6_IoU)
print("IoU for class4 is: ", class7_IoU)
print("IoU for class4 is: ", class8_IoU)


plt.imshow(train_images[0, :,:,0], cmap='gray')
plt.imshow(mask_images[0], cmap='gray')

In [ ]:
import random

test_img_number = random.randint(0, len(X_test))

test_img = X_test[test_img_number]
ground_truth = y_test[test_img_number]

test_img_norm = test_img[:,:,0][:,:,None]
test_img_input = np.expand_dims(test_img_norm, 0)

prediction = (model.predict(test_img_input))
predicted_img = np.argmax(prediction, axis=3)[0,:,:]
y_mapped = np.take(y_res2, predicted_img)

plt.figure(figsize=(12, 8))
plt.subplot(231)
plt.title('Tấm ảnh đầu vào')
plt.imshow(test_img[:,:,0], cmap='gray')
plt.subplot(232)
plt.title('Nhãn kiểm tra')
plt.imshow(ground_truth[:,:,0], cmap='gray')
plt.subplot(233)
plt.title('Kết quả dự đoán')
plt.imshow(y_mapped, cmap='gray')
plt.show()

In [ ]:
import random

test_img_number = random.randint(0, len(X_test))

test_img = X_test[test_img_number]
ground_truth = y_test[test_img_number]

test_img_norm = test_img[:,:,0][:,:,None]
test_img_input = np.expand_dims(test_img_norm, 0)

prediction = (model.predict(test_img_input))
predicted_img = np.argmax(prediction, axis=3)


plt.figure(figsize=(12, 8))
plt.subplot(231)
plt.title('Tấm ảnh đầu vào')
plt.imshow(test_img[:,:,0], cmap='gray')
plt.subplot(232)
plt.title('Nhãn kiểm tra')
plt.imshow(ground_truth[:,:,0], cmap='gray')
plt.subplot(233)
plt.title('Kết quả dự đoán')
plt.imshow(predicted_img, cmap='gray')
plt.show()

In [ ]:
# Apply a trained model on large image
from patchify import patchify, unpatchify

large_image = cv2.imread('large_images/large_image.tif', 0)
# This will split the image into small images of shape [3,3]
patches = patchify(large_image, (128, 128), step=128)  # Step=256 for 256 patches means no overlap

predicted_patches = []
for i in range(patches.shape[0]):
    for j in range(patches.shape[1]):
        print(i,j)
        
        single_patch = patches[i,j,:,:]       
        single_patch_norm = np.expand_dims(normalize(np.array(single_patch), axis=1),2)
        single_patch_input = np.expand_dims(single_patch_norm, 0)
        single_patch_prediction = (model.predict(single_patch_input))
        single_patch_predicted_img = np.argmax(single_patch_prediction, axis=3)[0,:,:]

        predicted_patches.append(single_patch_predicted_img)

predicted_patches = np.array(predicted_patches)
predicted_patches_reshaped = np.reshape(predicted_patches, (patches.shape[0], patches.shape[1], 128,128) )

reconstructed_image = unpatchify(predicted_patches_reshaped, large_image.shape)
plt.imshow(reconstructed_image, cmap='gray')
plt.imsave('imgs/seg.jpg', reconstructed_image, cmap='gray')

# Threshold everything above 0
# final_prediction = (reconstructed_image > 0.01).astype(np.uint8)
# plt.imshow(final_prediction)

In [ ]:
plt.hist(reconstructed_image.flatten())
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(221)
plt.title('Large Image')
plt.imshow(large_image, cmap='gray')
plt.subplot(222)
plt.title('Prediction of large Image')
plt.imshow(reconstructed_image, cmap='jet')
plt.show()